In [ ]:
!pip install unidecode selenium webdriver_manager
!sudo apt-get update
!sudo apt-get install chromium-driver -y
!apt-get install chromium-browser
!apt-get install -yq chromium-chromedriver
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
import datetime
import os
import random
import re
import requests
import time
import warnings
import numpy as np
import pandas as pd
import pickle
import regex
import tensorflow as tf
import threading
from bs4 import BeautifulSoup
from pytz import timezone
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from unidecode import unidecode
from webdriver_manager.chrome import ChromeDriverManager
from xgboost import XGBRegressor
import xgboost as xgb

import model
import evaluator
import date
import box_score
import helper_functions

warnings.filterwarnings("ignore")

eta = 0.1
cost_threshold = 0.5
expert_names = ['optimizer_preds', 'mlp_preds', 'dnn_preds', 'xgb_preds', 'lstm_preds', 'rf_preds']
num_experts = len(expert_names)
window_size = 10
past_x_games = 5
directory = '/NBA_Betting_Model/2023-24_Season/NBA-Bets-Multiplicative-Weights/' #directory to be changed
file_suffixes = ['Pts', 'Ast', 'trb', 'Pts+Ast', 'Pts+Rebs', 'Rebs+Ast', 'Pts+Rebs+Ast',
                 'Stl', 'Blk', 'Stl+Blk', 'fg3']
stats_dict = {
    'Pts': ['pts', 'fg3', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'usg_pct', 'off_rtg', 'bpm', 'mp'],
    'Ast': ['ast', 'ast_pct', 'usg_pct', 'off_rtg', 'bpm', 'mp'],
    'Rebs': ['trb', 'orb_pct', 'drb_pct', 'trb_pct', 'usg_pct', 'off_rtg', 'bpm', 'mp'],
    'Stl': ['stl', 'stl_pct', 'def_rtg', 'bpm', 'mp'],
    'Blk': ['blk', 'blk_pct', 'def_rtg', 'bpm', 'mp'],
    '3pt M': ['fg3', 'pts', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'usg_pct', 'off_rtg', 'bpm', 'mp']
}
combo_stats = ['Pts+Ast', 'Pts+Rebs', 'Rebs+Ast', 'Pts+Rebs+Ast', 'Stl+Blk']
models = ["DNN", "MLP", "RF", "XGB", "LSTM"]

start_date = datetime.date(2023, 10, 1)
end_date = datetime.date(2024, 6, 30)
date_range = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]

month_ends = {}
for date in date_range:
    month = date.month
    last_day = date.strftime('%d')
    if month < 10:
        month_ends[month] = int(str(month) + ('0' * (4 + (month - 1))) + str(int(last_day)))
    else:
        month_ends[month] = int(str(month) + last_day)
month_order = [10, 11, 12, 1, 2, 3, 4, 5, 6]
month_ends

In [ ]:
def load_files(directory, suffixes, prefix):
    data = {}
    for suffix in suffixes:
        with open(f'{directory}{prefix}{suffix}.pkl', 'rb') as f:
            data[suffix] = pickle.load(f)
    return data

import date
date_tracker = date.Date()
directory_month, directory_day = date_tracker.date_month_day(-2)
month, day = date_tracker.date_month_day(-1)
boxscore_month, boxscore_day = month, day
directory_base = f'{directory}{directory_month}/{directory_day}/'
expert_weights = load_files(directory_base, file_suffixes, 'expert_weights_')
expert_history = load_files(directory_base, file_suffixes, 'expert_history_')
date_counter = int(day) +1
month_order = month_order[month_order.index(int(month)):]
month = int(month)
last_day = 31
if month == 2:
    last_day = 28
elif month == 4 or month == 6:
    last_day = 30
if date_counter == 1:
    if month == 11:
        date_counter2 = month_ends[10]
    if month == 12:
        date_counter2 = month_ends[11]
    elif month == 1:
        date_counter2 = month_ends[12]
    else:
        date_counter2 = month_ends[month - 1]
else:
    if date_counter - 1 < 10:
        if month == 10:
            date_counter2 = int(str(100) + str(date_counter - 1))
        elif month == 11:
            date_counter2 = int(str(110) + str(date_counter - 1))
        elif month == 12:
            date_counter2 = int(str(120) + str(date_counter - 1))
        else:
            if date_counter - 1 == 9:
                date_counter2 = int(str(month) + ('0' * (5 + (month - 1))) + str(date_counter - 1))
            else:
                date_counter2 = int(str(month) + ('0' * (4 + (month - 1))) + str(date_counter - 1))
    else:
        if month == 10:
            date_counter2 = int(str(10) + str(date_counter - 1))
        elif month == 11:
            date_counter2 = int(str(11) + str(date_counter - 1))
        elif month == 12:
            date_counter2 = int(str(12) + str(date_counter - 1))
        else:
            date_counter2 = int(str(month) + ('0' * (4 + (month - 1))) + str(date_counter - 1))
date_counter -= 1
month_folder = os.path.join(directory, str(month))
if not os.path.exists(month_folder):
    os.makedirs(month_folder)
day_folder = os.path.join(month_folder, str(date_counter))
if not os.path.exists(day_folder):
    os.makedirs(day_folder)
model_predictions_dict = {}
if int(date_counter) < 10:
    date_counter = '0' + str(date_counter)

In [ ]:
month, date_counter, date_counter2

all box scores before yesterday

In [ ]:
import datetime
from datetime import datetime
from datetime import timedelta
date_format = '%m/%d/%Y %H:%M:%S %Z'
date = datetime.now(timezone('US/Pacific'))
date = date.astimezone(timezone('US/Pacific'))
today_date = date.strftime(date_format)
today = today_date.split('/')[0] + '/' + date.strftime(date_format).split('/')[1]
today_date = today_date.split(' ')[0]
month_name = datetime.strptime(today.split('/')[0].strip(), "%m").strftime("%B").lower()
month = today.split('/')[0]
day = today.split('/')[1]
action_date = month_name[:3].upper() + ' ' + day
time_delta = timedelta(days=1)
while not (month == '10' and day == '24'):
    date -= time_delta
    today_date = date.strftime(date_format)
    today = today_date.split('/')[0] + '/' + date.strftime(date_format).split('/')[1]
    month = today.split('/')[0]
    day = today.split('/')[1]
print(month, day)
print('before allboxscores')
all_box_score_results = pd.DataFrame()
while not (month == boxscore_month and day == boxscore_day):
    print(month, day)
    if (month, day) in [('11', '07'), ('11', '23'), ('12', '03'), ('12', '10')]:
        date += time_delta
        today_date = date.strftime(date_format)
        today = today_date.split('/')[0] + '/' + date.strftime(date_format).split('/')[1]
        month = today.split('/')[0]
        day = today.split('/')[1]
        print(month, day)
        continue
    past_box_score = pd.read_csv("/NBA_Betting_Model/2023-24_Season/NBA-Bets-Box-Score-Results/" + "NBA-Bets-Box-Score-Results-" + month + "-" + str(day.lstrip("0")) +".csv")
    past_box_score['date'] = month + "-" + day
    past_box_score['name'] = past_box_score['name'].apply(unidecode)
    past_box_score['name'] = past_box_score['name'].apply(helper_functions.abbrv)
    past_box_score['date'] = past_box_score['date'].apply(date_tracker.date_converter)
    all_box_score_results = all_box_score_results.append(past_box_score)
    all_box_score_results = all_box_score_results.dropna(subset=['date'])

    all_box_score_results['date'] = all_box_score_results['date'].astype('int')
    all_box_score_results = all_box_score_results.sort_values(by=['date'], ascending = False)
    all_box_score_results.set_index(['name','team']).index.factorize()[0]+1
    all_box_score_results = all_box_score_results.drop_duplicates(['date','name','team', 'opp', 'hmcrt_adv', 'pts', 'ast', 'trb'], keep='last')
    date += time_delta
    today_date = date.strftime(date_format)
    today = today_date.split('/')[0] + '/' + date.strftime(date_format).split('/')[1]
    month = today.split('/')[0]
    day = today.split('/')[1]

all_box_score_results

Yesterday's Box Score

In [ ]:

#extract yesterday's games statistics and add to cumulative statistics history
box_score = box_score.BoxScore()
box_score_result = box_score.full_box_scores(str(boxscore_month), str(boxscore_day))
box_score_result['date'] = boxscore_month + "-" + boxscore_day
print(boxscore_month + "-" + boxscore_day)
box_score_result['name'] = box_score_result['name'].apply(unidecode)
box_score_result['name'] = box_score_result['name'].apply(helper_functions.abbrv)
box_score_result['date'] = box_score_result['date'].apply(date_tracker.date_converter)
box_score_result

In [ ]:
boxscore_month, boxscore_day

In [ ]:
box_score_result.to_csv("/NBA_Betting_Model/2023-24_Season/NBA-Bets-Box-Score-Results/" + "NBA-Bets-Box-Score-Results-" + boxscore_month + "-" + str(boxscore_day.lstrip("0")) +".csv")

predictions = pd.read_csv('/NBA_Betting_Model/2023-24_Season/NBA-Bets-Predictions/NBA-Bets-' + str(boxscore_month) + '-' + str(boxscore_day) + '.csv')
predictions

Evaluate Yesterday's predictions

In [ ]:
evaluations = evaluator.Evaluator()
evaluated_predictions = evaluations.predictions_evaluator(predictions, box_score_result)
columns_to_update = ['All Game Percentages', 'All Game Correct', 'Last 5 Percentages', 'Last 5 Correct', 'Last 10 Percentages', 'Last 10 Correct']
predictions_subset = predictions[columns_to_update]
evaluated_predictions.update(predictions_subset)
evaluated_predictions

In [ ]:
evaluated_predictions.to_csv('/NBA_Betting_Model/2023-24_Season/NBA-Bets-Evaluations/NBA-Bets-Evaluations-' + str(boxscore_month) + '-' + str(boxscore_day) + '.csv')


In [ ]:
all_box_score_results_past_eval = all_box_score_results[all_box_score_results['mp'] > 1]
all_box_score_results_past_eval.insert(loc=0, column='player_id', value = all_box_score_results_past_eval.set_index(['name','team']).index.factorize()[0]+1)
all_box_score_results_past_eval = all_box_score_results_past_eval.sort_values(by = ['date', 'player_id'], ascending = [True, True])
all_box_score_results_past_eval

In [ ]:
def prop_type_conversion(prop_type):
    conversions = {'Rebs': 'trb', '3pt M': 'fg3'}
    return conversions.get(prop_type, prop_type)

def eval_past(folder_path, month, date_counter, prop_type, intersection, combined):
    past_eval = pd.read_csv('/NBA_Betting_Model/2023-24_Season/NBA-Bets-Evaluations/NBA-Bets-Evaluations-' + str(month) + '-' + str(date_counter) + '.csv')
    if prop_type == 'trb':
        prop_type = 'Rebs'
    if prop_type == 'fg3':
        prop_type = '3pt M'
    past_eval = past_eval.loc[past_eval['Play'].str.contains(prop_type, regex=False)]
    if '+' not in prop_type:
        past_eval = past_eval.loc[~past_eval['Play'].str.contains(r'\+')]
    if prop_type == 'Rebs+Ast':
        past_eval = past_eval.loc[~past_eval['Play'].str.contains('Pts')]
    elif prop_type == 'Pts+Rebs':
        past_eval = past_eval.loc[~past_eval['Play'].str.contains('Ast')]
    if not os.path.exists(folder_path + '/evals'):
        os.makedirs(folder_path + '/evals')
    unnamed_columns_past_eval = [col for col in past_eval.columns if col.startswith('Unnamed:')]
    past_eval = past_eval.drop(columns=unnamed_columns_past_eval)
    int_columns = ['Odds', 'Hmcrt_adv', 'All Game Correct', 'Last 5 Correct', 'Last 10 Correct']
    float_columns = ['Units', 'Payout', 'Profit', 'All Game Percentages', 'Last 5 Percentages', 'Last 10 Percentages']
    past_eval[int_columns] = past_eval[int_columns].apply(pd.to_numeric, errors='coerce')
    past_eval[float_columns] = past_eval[float_columns].astype(float)
    intersection_eval = pd.merge(intersection, past_eval)
    intersection_eval.to_csv(folder_path + '/evals/' + 'intersection_eval_' + str(prop_type) + '.csv')
    float_columns = ['Units', 'Payout', 'Profit', 'All Game Percentages', 'Last 5 Percentages', 'Last 10 Percentages', 'All Game Correct', 'Last 5 Correct', 'Last 10 Correct']
    past_eval[float_columns] = past_eval[float_columns].astype(float)
    combined_eval = pd.merge(combined, past_eval)
    combined_eval.to_csv(folder_path + '/evals/' + 'combined_eval_' + str(prop_type) + '.csv')
    return [1 if val == 'Y' else 0 for val in past_eval['Correct'].values]

def weighted_experts(expert_weights, expert_history, cost_threshold, num_experts, window_size, eta, all_preds, true_labels):
    decay_rate = 0.9
    for i in range(len(true_labels)):
        true_label = true_labels[i]
        expert_predictions = [preds[i] for preds in all_preds]
        weighted_predictions = expert_weights * expert_predictions
        total_weighted_prediction = np.sum(weighted_predictions)
        final_prediction = 1 if total_weighted_prediction > cost_threshold else 0
        for j in range(num_experts):
            expert_error = abs(expert_predictions[j] - true_label)
            expert_history[j, i % window_size] = expert_error
            decayed_eta = eta * (decay_rate ** i)
            expert_weights[j] *= np.exp(-decayed_eta * expert_error)
        expert_weights /= np.sum(expert_weights)
    return expert_weights

def prop_evaluation_with_experts(folder_path, month, date_counter, date_counter2, prop_type, features_set, past_data, past_x_games, expert_names, expert_weights_diff, expert_history_diff, cost_threshold, num_experts, window_size, eta, intersection, combined, passed_in_data1=None):
    prop_type = prop_type_conversion(prop_type)
    print('this is prop type')
    print(prop_type)
    pickle_path = os.path.join(folder_path, f'all_preds_{prop_type}.pkl')
    if os.path.exists(pickle_path):
        with open(pickle_path, 'rb') as f:
            all_preds = pickle.load(f)
    else:
        all_preds = None
    if all_preds:
        true_labels = eval_past(folder_path, month, date_counter, prop_type, intersection, combined)
        print(true_labels)
        expert_weights_diff = weighted_experts(expert_weights_diff, expert_history_diff, cost_threshold, num_experts, window_size, eta, all_preds, true_labels)
    else:
        expert_weights_diff = expert_weights_diff
    with open(os.path.join(folder_path, f'expert_weights_{prop_type}.pkl'), 'wb') as f:
        pickle.dump(expert_weights_diff, f)
    with open(os.path.join(folder_path, f'expert_history_{prop_type}.pkl'), 'wb') as f:
        pickle.dump(expert_history_diff, f)
    return expert_weights_diff, expert_history_diff

optimizations_folder_path = day_folder + '/optimizations'
final_combined_df = pd.read_csv(optimizations_folder_path + '/final_combined.csv')
final_intersection_df = pd.read_csv(optimizations_folder_path + '/final_intersection.csv')

for stat, attributes in stats_dict.items():
    if stat == 'Rebs':
        file_stat = 'trb'
    elif stat == '3pt M':
        file_stat = 'fg3'
    else:
        file_stat = stat
    expert_weights[stat], expert_history[stat] = prop_evaluation_with_experts(
        day_folder, month, date_counter, date_counter2, stat, attributes, all_box_score_results, past_x_games, expert_names,
        expert_weights[file_stat], expert_history[file_stat], cost_threshold, num_experts, window_size, eta, final_intersection_df, final_combined_df
    )

for combo in combo_stats:
    stats = [stat if stat != 'Rebs' else 'trb' for stat in combo.split('+')]
    expert_weights[combo], expert_history[combo] = prop_evaluation_with_experts(
        day_folder, month, date_counter, date_counter2, combo, [], all_box_score_results, past_x_games, expert_names, expert_weights[combo],
        expert_history[combo], cost_threshold, num_experts, window_size, eta, final_intersection_df, final_combined_df)


In [ ]:
evals_folder_path = day_folder + '/evals'
final_combined_df_eval = pd.DataFrame()
final_intersection_df_eval = pd.DataFrame()
for file in os.listdir(evals_folder_path):
    if file.startswith('combined_eval_'):
        df = pd.read_csv(os.path.join(evals_folder_path, file))
        final_combined_df_eval = pd.concat([final_combined_df_eval, df], ignore_index=True)
    elif file.startswith('intersection_eval_'):
        df = pd.read_csv(os.path.join(evals_folder_path, file))
        final_intersection_df_eval = pd.concat([final_intersection_df_eval, df], ignore_index=True)

unnamed_columns_final_combined_df_eval = [col for col in final_combined_df_eval.columns if col.startswith('Unnamed:')]
final_combined_df_eval = final_combined_df_eval.drop(columns=unnamed_columns_final_combined_df_eval)
final_combined_df_eval = final_combined_df_eval.drop_duplicates()
unnamed_columns_final_intersection_df_eval = [col for col in final_intersection_df_eval.columns if col.startswith('Unnamed:')]
final_intersection_df_eval = final_intersection_df_eval.drop(columns=unnamed_columns_final_intersection_df_eval)
final_intersection_df_eval = final_intersection_df_eval.drop_duplicates()
final_combined_df_eval.to_csv(evals_folder_path + '/final_combined_eval.csv', index=False)
final_intersection_df_eval.to_csv(evals_folder_path + '/final_intersection_eval.csv', index=False)


In [ ]:
final_combined_df_eval

In [ ]:
final_intersection_df_eval

In [ ]:
past_eval = pd.read_csv('/NBA_Betting_Model/2023-24_Season/NBA-Bets-Evaluations/NBA-Bets-Evaluations-' + str(boxscore_month) + '-' + str(boxscore_day) + '.csv')
past_eval

In [ ]:
optimized_curr = pd.read_csv('/NBA_Betting_Model/2023-24_Season/NBA-Bets-Optimized-Predictions/NBA-Optimized-Bets-' + str(boxscore_month) + '-' + str(boxscore_day) + '.csv')
optimized_curr

In [ ]:
def optimized_predictions_evaluator(optimized_predictions, current_evaluation):
    #return evaluations of past optimized predictions
    optimized_correct = []
    for i in range(len(optimized_predictions)):
        optimized_prediction = optimized_predictions.loc[i]
        if len(current_evaluation[(current_evaluation['Play'] == optimized_prediction['Play']) & (current_evaluation['Expert'] == optimized_prediction['Expert'])  & (current_evaluation['Odds'] == optimized_prediction['Odds'])]) == 0:
            optimized_correct.append('X')
        else:
            optimized_correct.append(current_evaluation[(current_evaluation['Play'] == optimized_prediction['Play']) & (current_evaluation['Expert'] == optimized_prediction['Expert'])  & (current_evaluation['Odds'] == optimized_prediction['Odds'])]['Correct'].values[0])
    return optimized_correct
optimized_predictions_evaluator(optimized_curr, past_eval)
optimized_evaluations = optimized_curr
optimized_evaluations['Correct'] = optimized_predictions_evaluator(optimized_curr, past_eval)
optimized_evaluations

In [ ]:
optimized_evaluations.to_csv("/NBA_Betting_Model/2023-24_Season/NBA-Bets-Optimized-Evaluations" + "NBA-Bets-Optimized-Evaluations-" + boxscore_month + "-" + boxscore_day +".csv")

NEW BET

In [ ]:
import date
date_tracker = date.Date()
month, day = date_tracker.date_month_day(0)
date_counter = int(day) +1
month_order = month_order[month_order.index(int(month)):]
month = int(month)
last_day = 31
if month == 2:
    last_day = 28
elif month == 4 or month == 6:
    last_day = 30
if date_counter == 1:
    if month == 11:
        date_counter2 = month_ends[10]
    if month == 12:
        date_counter2 = month_ends[11]
    elif month == 1:
        date_counter2 = month_ends[12]
    else:
        date_counter2 = month_ends[month - 1]
else:
    if date_counter - 1 < 10:
        if month == 10:
            date_counter2 = int(str(100) + str(date_counter - 1))
        elif month == 11:
            date_counter2 = int(str(110) + str(date_counter - 1))
        elif month == 12:
            date_counter2 = int(str(120) + str(date_counter - 1))
        else:
            if date_counter - 1 == 9:
                date_counter2 = int(str(month) + ('0' * (5 + (month - 1))) + str(date_counter - 1))
            else:
                date_counter2 = int(str(month) + ('0' * (4 + (month - 1))) + str(date_counter - 1))
    else:
        if month == 10:
            date_counter2 = int(str(10) + str(date_counter - 1))
        elif month == 11:
            date_counter2 = int(str(11) + str(date_counter - 1))
        elif month == 12:
            date_counter2 = int(str(12) + str(date_counter - 1))
        else:
            date_counter2 = int(str(month) + ('0' * (4 + (month - 1))) + str(date_counter - 1))
date_counter -= 1
month_folder = os.path.join(directory, str(month))
if not os.path.exists(month_folder):
    os.makedirs(month_folder)
day_folder = os.path.join(month_folder, str(date_counter))
if not os.path.exists(day_folder):
    os.makedirs(day_folder)
model_predictions_dict = {}
if int(date_counter) < 10:
    date_counter = '0' + str(date_counter)
month, date_counter, date_counter2

In [ ]:
def get_modified_html(initial_html, updated_html):
    initial_soup = BeautifulSoup(initial_html, "html.parser")
    updated_soup = BeautifulSoup(updated_html, "html.parser")
    modified_parts = []
    for initial_tag, updated_tag in zip(initial_soup.find_all(), updated_soup.find_all()):
        if str(initial_tag) != str(updated_tag):
            modified_parts.append(updated_tag)
    modified_html = '\n'.join(str(tag) for tag in modified_parts)
    return modified_html

def click_with_retry(driver, entry, selector, max_attempts=5):
    attempts = 0
    while attempts < max_attempts:
        try:
            see_all_button = entry.find_element(By.CSS_SELECTOR, selector)
            driver.execute_script("arguments[0].click();", see_all_button)
            return
        except selenium.common.exceptions.StaleElementReferenceException:
            attempts += 1
            time.sleep(1)

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_options.binary_location = "/usr/bin/chromium-browser"
driver = webdriver.Chrome(options=chrome_options)

import datetime
from datetime import datetime
from datetime import timedelta
date_format = '%m/%d/%Y %H:%M:%S %Z'
date = datetime.now(timezone('US/Pacific'))
date = date.astimezone(timezone('US/Pacific'))
today_date = date.strftime(date_format)
today = today_date.split('/')[0] + '/' + date.strftime(date_format).split('/')[1]
today_date = today_date.split(' ')[0]
month_name = datetime.strptime(today.split('/')[0].strip(), "%m").strftime("%B").lower()
action_month = today.split('/')[0]
action_day = today.split('/')[1]
action_date = month_name[:3].upper() + ' ' + action_day
action_month, action_day

In [ ]:
def prop_type_conversion(prop_type):
    conversions = {'Rebs': 'trb', '3pt M': 'fg3'}
    return conversions.get(prop_type, prop_type)

def process_csv(filename, prop_type):
    data = pd.read_csv(filename)
    data = data.loc[data['Play'].str.contains(prop_type, regex=False)]
    if '+' not in prop_type:
        data = data.loc[~data['Play'].str.contains(r'\+')]
    if prop_type == 'Rebs+Ast':
        data = data.loc[~data['Play'].str.contains('Pts')]
    elif prop_type == 'Pts+Rebs':
        data = data.loc[~data['Play'].str.contains('Ast')]
    return data

def weighted_optimizer(folder_path, expert_names, expert_weights, prop_type, all_predictions_dfs):
    expert_to_df_map = {expert_name: all_predictions_dfs[i] for i, expert_name in enumerate(expert_names)}
    if len(set(expert_weights)) == 1:
        best_experts = random.sample(expert_names, 2)
    else:
        indices = np.array(expert_weights).argsort()[-2:][::-1]
        best_experts = [expert_names[index] for index in indices]
    major1 = expert_to_df_map[best_experts[0]]
    major2 = expert_to_df_map[best_experts[1]]
    if not os.path.exists(folder_path + '/optimizations'):
        os.makedirs(folder_path + '/optimizations')
    expert_to_df_map = {expert_name: df for expert_name, df in zip(expert_names, all_predictions_dfs)}
    if len(set(expert_weights)) == 1:
        best_experts = random.sample(expert_names, 2)
    else:
        indices = np.array(expert_weights).argsort()[-2:][::-1]
        best_experts = [expert_names[index] for index in indices]
    major1 = expert_to_df_map[best_experts[0]]
    major2 = expert_to_df_map[best_experts[1]]
    unnamed_columns_major1 = [col for col in major1.columns if col.startswith('Unnamed:')]
    major1 = major1.drop(columns=unnamed_columns_major1)
    unnamed_columns_major2 = [col for col in major2.columns if col.startswith('Unnamed:')]
    major2 = major2.drop(columns=unnamed_columns_major2)
    int_columns = ['Odds', 'Hmcrt_adv']
    float_columns = ['Units', 'Payout', 'Profit']
    major1[int_columns] = major1[int_columns].apply(pd.to_numeric, errors='coerce')
    major1[float_columns] = major1[float_columns].astype(float)
    major2[int_columns] = major2[int_columns].apply(pd.to_numeric, errors='coerce')
    major2[float_columns] = major2[float_columns].astype(float)
    intersection = pd.merge(major1, major2, how='inner').drop_duplicates()
    intersection.to_csv(folder_path + '/optimizations/' + 'intersection_' + str(prop_type) + '.csv')
    combined = pd.concat([major1, major2], axis=0).drop_duplicates()
    combined.to_csv(folder_path + '/optimizations/' + 'combined_' + str(prop_type) + '.csv')
    return intersection, combined

def get_model_names_from_predictions(predictions, prop_type):
    return [col.split('_')[1] for col in predictions.columns if f"PlayerPredictions_" in col and prop_type in col]

def get_prediction(predictions, model_name, prop_type, player_name, team):
    filtered_df = predictions[(predictions['PlayerName'] == player_name) &
                              (predictions['PlayerTeam'] == team)]
    if not filtered_df.empty:
        return float(filtered_df[f'PlayerPredictions_{model_name}_{prop_type}'].values[0][0])
    else:
        return 0.0

def models_predictions_eval(folder_path, predictions, optimized, unoptimized, prop_type):
    model_names = get_model_names_from_predictions(predictions, prop_type)
    preds_dict = {name: [] for name in model_names}
    optimizer_preds = []
    for play, team in zip(unoptimized['Play'].values, unoptimized['Teams'].values):
        player_name = play.split(' ')[0]
        bet_value = float(play.split(' ')[1][1:])
        over_under = play.split(' ')[1][0]
        if play not in optimized['Play'].values:
            optimizer_preds.append(0)
        else:
            optimizer_preds.append(1)
        for model_name in model_names:
            prediction = get_prediction(predictions, model_name, prop_type, player_name, team)
            is_over = int(prediction > bet_value)
            is_under = int(prediction < bet_value)
            preds_dict[model_name].append(is_over if over_under == 'o' else is_under)
    prediction_path = os.path.join(folder_path, 'predictions')
    if not os.path.exists(prediction_path):
        os.makedirs(prediction_path)
    predictions_df_dict = {}
    for model_name in model_names:
        predictions_df_dict[model_name] = unoptimized[[bool(x) for x in preds_dict[model_name]]]
        predictions_df_dict[model_name].to_csv(os.path.join(prediction_path, f'{model_name}_{prop_type}.csv'))
    all_preds = [optimizer_preds] + list(preds_dict.values())
    all_predictions_dfs = [optimized] + list(predictions_df_dict.values())
    return all_preds, all_predictions_dfs

def preprocess_model(past_data, prop_type, features_set):
    info_data = past_data[features_set]
    info_data = info_data.dropna()
    features = list(info_data.columns)
    features.remove(prop_type.lower())
    target = [prop_type.lower()]
    X = info_data.loc[:, features].values
    y = info_data.loc[:, target].values
    return train_test_split(X, y, test_size=0.1, random_state=1)

def train_model(model, X_train, y_train):
    model.fit(X_train, y_train)

def predict_model(model, player_last_avgs, player_preds):
    with open(os.devnull, 'w') as devnull:
        # sys.stdout = devnull
        player_pred = model.predict(player_last_avgs)
        # sys.stdout = sys.__stdout__
        player_preds.append(player_pred)

def define_models(X_train, y_train):
    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer.adapt(np.array(X_train))
    model_defs = {
        'DNN': Sequential([
            normalizer,
            Dense(128, activation='relu'),
            Dense(128, activation='relu'),
            Dense(1)
        ]),
        'MLP': MLPRegressor(hidden_layer_sizes=(100,), solver='adam', random_state=42, max_iter=5000),
        'RF': RandomForestRegressor(n_estimators=200, random_state=42),
        'XGB': XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.05,
                            max_depth=8, subsample=0.9, colsample_bytree=0.9, random_state=42, verbosity=0),
        'LSTM': Sequential([
            LSTM(128, input_shape=(X_train.shape[1], 1)),
            Dense(1)
        ])
    }
    model_defs['DNN'].compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
    model_defs['LSTM'].compile(optimizer='adam', loss='mse')
    threads = {name: threading.Thread(target=train_model, args=(model, X_train, y_train)) for name, model in model_defs.items()}
    [thread.start() for thread in threads.values()]
    [thread.join() for thread in threads.values()]
    return model_defs

def last_x_avgs(data, prop_type, features_set, x):
    last_x_games = data[-x:]
    last_x_stats = last_x_games[features_set].mean(axis=0).to_frame().T
    last_x_stats = last_x_stats.drop(prop_type, axis=1)
    return last_x_stats

def models_eval(past_data, past_x_games, prop_type, features_set, trained_models):
    player_data_list = []
    player_preds = {name: [] for name in trained_models}
    for i in past_data['player_id'].unique():
        player_data = past_data[past_data['player_id'] == i].fillna(0)
        player_last_avgs = last_x_avgs(player_data, prop_type.lower(), features_set, past_x_games)
        player_data_list.append({
            'PlayerName': player_data['name'].values[0],
            'PlayerTeam': player_data['team'].values[0],
            'PlayerLastAvgs': player_last_avgs
        })
    for pdata in player_data_list:
        threads = {model_name: threading.Thread(target=predict_model, args=(model, pdata['PlayerLastAvgs'], player_preds[model_name]))
                   for model_name, model in trained_models.items()}
        [thread.start() for thread in threads.values()]
        [thread.join() for thread in threads.values()]
    predictions = pd.DataFrame({
        'PlayerName': [pdata['PlayerName'] for pdata in player_data_list],
        'PlayerTeam': [pdata['PlayerTeam'] for pdata in player_data_list],
        **{f'PlayerPredictions_{name}_{prop_type}': preds for name, preds in player_preds.items()}
    })
    return predictions

def prop_prediction_with_experts(folder_path, month, date_counter, date_counter2, prop_type, features_set, past_data, past_x_games, expert_names, expert_weights_diff, expert_history_diff, cost_threshold, num_experts, window_size, eta, passed_in_data1=None):
    prop_type = prop_type_conversion(prop_type)
    print('this is prop type')
    print(prop_type)
    if passed_in_data1 is not None:
        models_predictions = passed_in_data1
    elif '+' in prop_type:
        models_predictions = passed_in_data1
    else:
        X_train, X_val, y_train, y_val = preprocess_model(past_data, prop_type, features_set)
        trained_models = define_models(X_train.astype(float), y_train.astype(float))
        models_predictions = models_eval(past_data, past_x_games, prop_type, features_set, trained_models)
    return models_predictions

for stat, attributes in stats_dict.items():
    if stat == 'Rebs':
        file_stat = 'trb'
    elif stat == '3pt M':
        file_stat = 'fg3'
    else:
        file_stat = stat
    model_predictions_dict[stat] = prop_prediction_with_experts(
        day_folder, month, date_counter, date_counter2, stat, attributes, all_box_score_results_past_eval, past_x_games, expert_names,
        expert_weights[file_stat], expert_history[file_stat], cost_threshold, num_experts, window_size, eta
    )
key_mapping = {
    'Rebs': 'trb',
    '3pt M': 'fg3'
}
model_predictions_dict = {
    key_mapping.get(key, key): value for key, value in model_predictions_dict.items()
}
for combo in combo_stats:
    stats = [stat if stat != 'Rebs' else 'trb' for stat in combo.split('+')]
    if all([not model_predictions_dict[stat].empty for stat in stats]):
        merged_predictions = model_predictions_dict[stats[0]]
        for stat in stats[1:]:
            merged_predictions = pd.merge(merged_predictions, model_predictions_dict[stat], on=['PlayerName', 'PlayerTeam'], how='inner')
        for model in models:
            merged_predictions[f'PlayerPredictions_{model}_{combo}'] = sum([merged_predictions[f'PlayerPredictions_{model}_{stat}'] for stat in stats])
        model_predictions_dict[combo] = prop_prediction_with_experts(
            day_folder, month, date_counter, date_counter2, combo, [], all_box_score_results_past_eval, past_x_games, expert_names, expert_weights[combo],
            expert_history[combo], cost_threshold, num_experts, window_size, eta, merged_predictions)

matchups

In [ ]:
def site_scrape_chrome(url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--start-maximized')
    chrome_options.add_argument('--disable-infobars')
    chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
    chrome_options.binary_location = "/usr/bin/chromium-browser"
    driver = webdriver.Chrome(options=chrome_options)
    tz_params = {'timezoneId': 'America/Los_Angeles'}
    driver.execute_cdp_cmd('Emulation.setTimezoneOverride', tz_params)
    driver.get(url)
    time.sleep(5)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    driver.close()
    return soup

if int(action_month) > 9:
    if int(action_day) < 10:
        soup = site_scrape_chrome('https://theathletic.com/nba/schedule/2023-' + action_month + '-' + action_day + '/')
    else:
        soup = site_scrape_chrome('https://theathletic.com/nba/schedule/2023-' + action_month + '-' + action_day + '/')
else:
    if int(action_day) < 10:
        soup = site_scrape_chrome('https://theathletic.com/nba/schedule/2023-' + action_month + '-' + action_day + '/')
    else:
        soup = site_scrape_chrome('https://theathletic.com/nba/schedule/2023-' + action_month + '-' + action_day + '/')
games = soup.select('tr.MuiTableRow-root .jss6')
matchups = []
if games:
    for i in range(0, len(games), 2):
        home_team = str(games[i]).split('>')[2].split('<')[0]
        away_team = str(games[i+1]).split('>')[2].split('<')[0]
        matchup = f"{home_team} vs {away_team}"
        matchups.append(matchup)
else:
    for val in str(soup).split('@context":"http://www.schema.org"')[1:]:
        match = re.search(r'"name":"([^"]+ @ [^"]+)"', val)
        if match:
            matchup = match.group(1)
            teams = matchup.split(' @ ')
            if len(teams) == 2:
                away_team, home_team = teams
                matchup = f"{home_team} vs {away_team}"
                matchups.append(matchup)
            else:
                break
        else:
            break
matchups

ACTION Bets

In [ ]:
def calculate_payout(row):
    odds = row['Odds']
    units = row['Units']
    if int(odds) > 0:
        multiplier = (int(odds) / 100) + 1
    else:
        multiplier = (-100 / int(odds)) + 1
    return float(float(multiplier) * float(units))
tz_params = {'timezoneId': 'America/Los_Angeles'}
driver.execute_cdp_cmd('Emulation.setTimezoneOverride', tz_params)
driver.get('https://www.actionnetwork.com/nba/picks')
time.sleep(5)
initial_html = driver.page_source
modified_htmls = []
entries = driver.find_elements(By.CSS_SELECTOR, '.css-1pfshtc.e11h3jb20')
for entry in entries:
    see_all_button = entry.find_element(By.CSS_SELECTOR, 'button[data-testid="expert-picks__see-all-picks"]')
    click_with_retry(driver, entry, 'button[data-testid="expert-picks__see-all-picks"]')
    time.sleep(5)
    updated_html = driver.page_source
    modified_html = get_modified_html(initial_html, updated_html)
    modified_htmls.append(modified_html)
play_list, expert_list, odds_list, units_list, name_list = [], [], [], [], []
site_data = modified_htmls
for html in site_data:
    for picks in str(html).split('Player Props')[1].split('<div class="pick-card__header">')[1:]:
        soup = BeautifulSoup(picks, 'html.parser')
        play_elements = soup.select('.base-pick__name')
        odds_elements = soup.select('.base-pick__secondary-text')
        units_elements = soup.select('.base-pick__units')
        expert_element = soup.select_one('.pick-card__expert-info > a')
        if expert_element:
            expert = expert_element['href'].split('/')[-1]
        else:
            expert = ''
        for play_element, odds_element, units_element in zip(play_elements, odds_elements, units_elements):
            if re.match(r'[A-Z]\.[A-Za-z]+ [ou][\d]+\.[\d] [A-Za-z]+', play_element.text):
                play_list.append(play_element.text)
                expert_list.append(expert)
                odds_list.append(odds_element.text if odds_element else '')
                units_list.append(units_element.text.split('u')[0] if units_element else '')
                name_list.append(str(play_element.text).split(' ')[0])
data = {
    'Play': play_list,
    'Expert': expert_list,
    'Odds': odds_list,
    'Units': units_list,
    'Name': name_list
}
bets = pd.DataFrame(data)
bets.to_csv("/NBA_Betting_Model/2023-24_Season/NBA-Bets-Predictions/" + "NBA-Bets-" + str(action_month) + '-' + str(action_day) + '.csv')
bets = bets[~bets['Play'].str.contains('TOs', na=False)]
bets = bets[~bets['Play'].str.contains('(Live)', na=False)]
bets = bets.reset_index(drop=True)

all_box_score_results_for_predictions = all_box_score_results.sort_values(by='date', ascending=False)

names, set_teams, opponents, hmcrt_advantages = [], [], [], []
all_teams = set(all_box_score_results_for_predictions['team'].values)
for i in range(len(bets)):
    bet = bets.loc[i]
    name = bet['Name'].split(' ')[0]
    matching_name = all_box_score_results_for_predictions[all_box_score_results_for_predictions['name'] == name]
    if len(set(matching_name['name'].values)) > 1:
        set_teams.append('')
        names.append(np.NAN)
        opponents.append(np.NAN)
        hmcrt_advantages.append(np.NAN)
    elif len(set(matching_name['name'].values)) < 1:
        print('less than 1')
        set_teams.append('')
        names.append(np.NAN)
        opponents.append(np.NAN)
        hmcrt_advantages.append(np.NAN)
    else:
        #gets most recent team regardless
        team = matching_name.iloc[0]['team']
        found_today = False
        for matchup in matchups:
            matchup_home = matchup.split('vs')[0].strip()
            matchup_away = matchup.split('vs')[1].strip()
            if matchup_home in team:
                set_teams.append(team)
                names.append(name)
                for t in all_teams:
                    if matchup_away in t:
                        opponents.append(t)
                        break
                hmcrt_advantages.append(1)
                found_today = True
                break
            elif matchup_away in team:
                set_teams.append(team)
                names.append(name)
                for t in all_teams:
                    if matchup_home in t:
                        opponents.append(t)
                        break
                hmcrt_advantages.append(0)
                found_today = True
                break
        if found_today:
            continue
        else:
            set_teams.append('')
            names.append(np.NAN)
            opponents.append(np.NAN)
            hmcrt_advantages.append(np.NAN)
bets['Name'] = names
bets['Teams'] = set_teams
bets['Opponent'] = opponents
bets['Hmcrt_adv'] = hmcrt_advantages
bets['Payout'] = bets.apply(calculate_payout, axis=1)
bets['Payout'] = bets['Payout'].astype(float)
bets['Units'] = bets['Units'].astype(float)
bets['Profit'] = bets['Payout'] - bets['Units']
bets = bets[bets['Name'].notna()]
bets = bets.drop_duplicates(subset=['Play', 'Expert'], keep='first').reset_index(drop=True)
bets

In [ ]:
def all_evaluations_func(cur_month, cur_day):
    #return cumulative evaluations
    all_evals = pd.DataFrame()
    import datetime
    from datetime import datetime
    from datetime import timedelta
    date_format = '%m/%d/%Y %H:%M:%S %Z'
    date = datetime.now(timezone('US/Pacific'))
    date = date.astimezone(timezone('US/Pacific'))
    today_date = date.strftime(date_format)
    today = today_date.split('/')[0] + '/' + date.strftime(date_format).split('/')[1]
    today_date = today_date.split(' ')[0]
    month_name = datetime.strptime(today.split('/')[0].strip(), "%m").strftime("%B").lower()
    month = today.split('/')[0]
    day = today.split('/')[1]
    action_date = month_name[:3].upper() + ' ' + day
    time_delta = timedelta(days=1)
    while not (month == '10' and day == '24'):
        date -= time_delta
        today_date = date.strftime(date_format)
        today = today_date.split('/')[0] + '/' + date.strftime(date_format).split('/')[1]
        month = today.split('/')[0]
        day = today.split('/')[1]
    print(month, day)
    print('before allevals')
    while not (month == cur_month and day == cur_day):
        if (month, day) in [('11', '07'), ('11', '23'), ('12', '03'), ('12', '10'), ('12', '24')]:
          date += time_delta
          today_date = date.strftime(date_format)
          today = today_date.split('/')[0] + '/' + date.strftime(date_format).split('/')[1]
          month = today.split('/')[0]
          day = today.split('/')[1]
          print(month, day)
          continue
        eval_ = pd.read_csv("/NBA_Betting_Model/2023-24_Season/NBA-Bets-Evaluations/" + "NBA-Bets-Evaluations-" + month + "-" + str(day) +".csv")
        all_evals = all_evals.append(eval_)
        date += time_delta
        today_date = date.strftime(date_format)
        today = today_date.split('/')[0] + '/' + date.strftime(date_format).split('/')[1]
        month = today.split('/')[0]
        day = today.split('/')[1]
        print(month, day)
    print('broke')
    print(month, day)
    return all_evals

import model
model = model.Model()
import evaluator
evaluations = evaluator.Evaluator()
all_evaluations = all_evaluations_func(str(action_month), str(action_day))
all_evaluations = all_evaluations.rename(columns={'Opponent': 'opponent', 'Name': 'name'})
all_evaluations = model.preprocessing(all_evaluations, True)
all_evaluations


In [ ]:
all_box_score_results_for_evaluations = all_box_score_results_for_predictions
unoptimized = bets
bets_inputs = evaluations.past_games_trends(unoptimized, all_box_score_results_for_evaluations, False)
unoptimized.to_csv(f'/NBA_Betting_Model/2023-24_Season/NBA-Bets-Predictions/NBA-Bets-{action_month}-{action_day}.csv')

In [ ]:
#generating the optimized predictions parts now
bets_inputs = bets_inputs.rename(columns={'Opponent': 'opponent', 'Name': 'name'})
bets_inputs = model.preprocessing(bets_inputs, False)
features = model.features_processing(list(all_evaluations.columns[:len(all_evaluations.columns) - 1])[5:])
features = [item for item in features if "Unnamed" not in item]
target = [all_evaluations.columns[-1]]
X, Y, final_model = model.train(features, target, all_evaluations)
final_model.fit(X, Y)
for i in set(features).difference(set(bets_inputs.columns[:len(bets_inputs.columns) - 1])):
    bets_inputs[i] = 0

#model predicts to optimize from today's set of predictions
X_predictions = bets_inputs.loc[:, features].values
predictions = final_model.predict(X_predictions)
indices = [i for i in range(len(predictions)) if predictions[i] == 'Y']
unoptimized.iloc[indices, :]

In [ ]:
optimized_file_path = f'/NBA_Betting_Model/2023-24_Season/NBA-Bets-Optimized-Predictions/NBA-Optimized-Bets-{action_month}-{action_day}.csv'
unoptimized.iloc[indices, :].to_csv(optimized_file_path)

In [ ]:
action_month, action_day

In [ ]:
day_folder

In [ ]:
def prop_prediction_with_experts_eval(folder_path, month, date_counter, prop_type, expert_weights_diff, models_predictions):
    unoptimized = process_csv(f'/NBA_Betting_Model/2023-24_Season/NBA-Bets-Predictions/NBA-Bets-{month}-{date_counter}.csv', prop_type)
    optimized = process_csv(f'/NBA_Betting_Model/2023-24_Season/NBA-Bets-Optimized-Predictions/NBA-Optimized-Bets-{month}-{date_counter}.csv', prop_type)
    prop_type = prop_type_conversion(prop_type)
    print('this is prop type')
    print(prop_type)
    all_preds = None
    all_preds, all_predictions_dfs = models_predictions_eval(folder_path, models_predictions, optimized, unoptimized, prop_type)
    all_empty = True
    for df in all_predictions_dfs:
        if not df.empty:
            all_empty = False
            break
    if not all_empty:
      intersection, combined = weighted_optimizer(folder_path, expert_names, expert_weights_diff, prop_type, all_predictions_dfs)
      prop_type = prop_type.replace('\\', '')
      if all_preds is not None:
          with open(os.path.join(folder_path, f'all_preds_{prop_type}.pkl'), 'wb') as f:
              pickle.dump(all_preds, f)
    return models_predictions

for stat, attributes in stats_dict.items():
    if stat == 'Rebs':
        file_stat = 'trb'
    elif stat == '3pt M':
        file_stat = 'fg3'
    else:
        file_stat = stat
    model_predictions_dict[stat] = prop_prediction_with_experts_eval(
        day_folder, action_month, action_day, stat, expert_weights[file_stat], model_predictions_dict[file_stat])
model_predictions_dict = {
    key_mapping.get(key, key): value for key, value in model_predictions_dict.items()
}
for combo in combo_stats:
    stats = [stat if stat != 'Rebs' else 'trb' for stat in combo.split('+')]
    if all([not model_predictions_dict[stat].empty for stat in stats]):
        merged_predictions = model_predictions_dict[stats[0]]
        for stat in stats[1:]:
            merged_predictions = pd.merge(merged_predictions, model_predictions_dict[stat], on=['PlayerName', 'PlayerTeam'], how='inner')
        for model in models:
            merged_predictions[f'PlayerPredictions_{model}_{combo}'] = sum([merged_predictions[f'PlayerPredictions_{model}_{stat}'] for stat in stats])
        model_predictions_dict[combo] = prop_prediction_with_experts_eval(
        day_folder, action_month, action_day, combo, expert_weights[combo], model_predictions_dict[combo])

optimizations_folder_path = day_folder + '/optimizations'
final_combined_df = pd.DataFrame()
final_intersection_df = pd.DataFrame()
for file in os.listdir(optimizations_folder_path):
    if file.startswith('combined_'):
        df = pd.read_csv(os.path.join(optimizations_folder_path, file))
        final_combined_df = pd.concat([final_combined_df, df], ignore_index=True)
    elif file.startswith('intersection_'):
        df = pd.read_csv(os.path.join(optimizations_folder_path, file))
        final_intersection_df = pd.concat([final_intersection_df, df], ignore_index=True)
final_combined_df = final_combined_df.drop_duplicates()
final_intersection_df = final_intersection_df.drop_duplicates()
final_combined_df.to_csv(optimizations_folder_path + '/final_combined.csv', index=False)
final_intersection_df.to_csv(optimizations_folder_path + '/final_intersection.csv', index=False)

In [ ]:
final_combined_df

In [ ]:
final_intersection_df

In [ ]:
optimized = unoptimized.iloc[indices, :]
optimized

In [ ]:
final_combined_df = final_combined_df.loc[:, ~final_combined_df.columns.str.contains('Unnamed')]
final_intersection_df = final_intersection_df.loc[:, ~final_intersection_df.columns.str.contains('Unnamed')]
optimized = optimized.loc[:, ~optimized.columns.str.contains('Unnamed')]
merge_columns = ['Play', 'Expert', 'Odds', 'Name', 'Teams', 'Opponent', 'Hmcrt_adv']
for column in merge_columns:
    final_combined_df[column] = final_combined_df[column].astype(str)
    final_intersection_df[column] = final_intersection_df[column].astype(str)
    optimized[column] = optimized[column].astype(str)
overall_intersection_df = pd.merge(final_combined_df, final_intersection_df, on=merge_columns, how='inner')
overall_intersection_df = pd.merge(overall_intersection_df, optimized, on=merge_columns, how='inner')
overall_intersection_df_sorted_df = overall_intersection_df.sort_values(by='All Game Percentages', ascending=False)
overall_intersection_df_sorted_df